In [ ]:
#import findspark
import pyspark

#findspark.init()

spark = pyspark.sql.SparkSession.builder \
        .master('local') \
        .appName('Aggregates with PySpark') \
        .getOrCreate()

sc = spark.sparkContext

# Aggregates

In this notebook we will be using data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Online%20Retail).  
We'll actually use [this version](https://www.kaggle.com/carrie1/ecommerce-data) of the dataset for convenience reasons (csv format).

In [ ]:
from pathlib import Path
INPUT_PATH = Path(*['..'] * 3, 'input', 'Online-Retail-Dataset')
filename = 'data.csv'
filepath = INPUT_PATH / filename
filepath

Out[2]: PosixPath('../../../input/Online-Retail-Dataset/data.csv')

In [ ]:
sales = spark.read.format("csv").options(inferSchema=True, header=True).load("dbfs:/FileStore/shared_uploads/thibaudchevrier@gmail.com/data.csv")
sales.printSchema()
print("Shape:", (sales.count(), len(sales.columns)))
sales.show(5)

root
-- InvoiceNo: string (nullable = true)
-- StockCode: string (nullable = true)
-- Description: string (nullable = true)
-- Quantity: integer (nullable = true)
-- InvoiceDate: string (nullable = true)
-- UnitPrice: double (nullable = true)
-- CustomerID: integer (nullable = true)
-- Country: string (nullable = true)

Shape: (541909, 8)
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
 536365| 85123A|WHITE HANGING HEA...| 6|12/1/2010 8:26| 2.55| 17850|United Kingdom|
 536365| 71053| WHITE METAL LANTERN| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom|
 536365| 84406B|CREAM CUPID HEART...| 8|12/1/2010 8:26| 2.75| 17850|United Kingdom|
 536365| 84029G|KNITTED UNION FLA...| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom|
 536365| 84029E|RED WOOLLY HOTTIE...| 6|12/1/2010 8:26| 3.39| 17850|United Kingdom|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
only showing top 5 rows

Before we get started, let's import PySpark's SQL functions.

In [ ]:
from pyspark.sql import functions as F

We're **ready to roll!**

What's the type of `sales.groupBy('CustomerID')`?

In [ ]:
type(sales.groupBy('CustomerID'))

Out[7]: pyspark.sql.group.GroupedData

A `GroupedData`, here's the link to the [documentation](https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.GroupedData).

We'll go through what we can do with this.

Compute the average of all numeric columns with `.avg()`

In [ ]:
sales.groupBy('CustomerID').avg().show(5)

+----------+------------------+------------------+---------------+
CustomerID| avg(Quantity)| avg(UnitPrice)|avg(CustomerID)|
+----------+------------------+------------------+---------------+
 17420| 8.833333333333334| 3.881333333333332| 17420.0|
 16861| 16.625|2.3412500000000005| 16861.0|
 16503| 6.593023255813954| 4.198604651162791| 16503.0|
 15727|10.149006622516556|3.2481456953642383| 15727.0|
 17389|33.169642857142854|5.6855803571428565| 17389.0|
+----------+------------------+------------------+---------------+
only showing top 5 rows

`.mean()` is an alias to `.avg()`

In [ ]:
sales.groupBy('CustomerID').mean().show()

+----------+------------------+------------------+---------------+
CustomerID| avg(Quantity)| avg(UnitPrice)|avg(CustomerID)|
+----------+------------------+------------------+---------------+
 17420| 8.833333333333334| 3.881333333333332| 17420.0|
 16861| 16.625|2.3412500000000005| 16861.0|
 16503| 6.593023255813954| 4.198604651162791| 16503.0|
 15727|10.149006622516556|3.2481456953642383| 15727.0|
 17389|33.169642857142854|5.6855803571428565| 17389.0|
 15447| 9.444444444444445|3.1211111111111114| 15447.0|
 14450| 6.025|2.7905000000000006| 14450.0|
 15100| 9.666666666666666|10.950000000000001| 15100.0|
 12471| 15.0| 4.269058380414311| 12471.0|
 16916| 3.076923076923077| 2.22041958041958| 16916.0|
 14514| 8.068493150684931|3.2264383561643846| 14514.0|
 16500|11.181818181818182| 17.12| 16500.0|
 12626|10.912052117263844| 2.840390879478826| 12626.0|
 18161| 10.0| 3.631604938271605| 18161.0|
 15967| 2.511904761904762|2.8676190476190477| 15967.0|
 17809| 31.484375|14.018593750000003| 17809.0|
 15738|16.857954545454547| 2.118295454545455| 15738.0|
 17044| 48.7|3.8419999999999996| 17044.0|
 12393| 12.75| 2.2796875| 12393.0|
 16791| 2.357142857142857| 2.928975155279503| 16791.0|
+----------+------------------+------------------+---------------+
only showing top 20 rows

`.sum()`

In [ ]:
sales.groupBy('customerID').sum().show(5)

+----------+-------------+------------------+---------------+
customerID|sum(Quantity)| sum(UnitPrice)|sum(CustomerID)|
+----------+-------------+------------------+---------------+
 17420| 265|116.43999999999997| 522600|
 16861| 133|18.730000000000004| 134888|
 16503| 567| 361.08| 1419258|
 15727| 3065| 980.9399999999999| 4749554|
 17389| 7430| 1273.57| 3895136|
+----------+-------------+------------------+---------------+
only showing top 5 rows

We can select a specific column

In [ ]:
sales.groupBy('customerID').mean('Quantity').show(5)

+----------+------------------+
customerID| avg(Quantity)|
+----------+------------------+
 17420| 8.833333333333334|
 16861| 16.625|
 16503| 6.593023255813954|
 15727|10.149006622516556|
 17389|33.169642857142854|
+----------+------------------+
only showing top 5 rows

or more

In [ ]:
sales.groupBy('customerID').mean('Quantity', 'UnitPrice').show(5)

+----------+------------------+------------------+
customerID| avg(Quantity)| avg(UnitPrice)|
+----------+------------------+------------------+
 17420| 8.833333333333334| 3.881333333333332|
 16861| 16.625|2.3412500000000005|
 16503| 6.593023255813954| 4.198604651162791|
 15727|10.149006622516556|3.2481456953642383|
 17389|33.169642857142854|5.6855803571428565|
+----------+------------------+------------------+
only showing top 5 rows

Won't work with a list, a tuple or a generator.  
These need to be ***unpacked**.

In [ ]:
# This will fail without unpacking
sales.groupBy('customerID').sum(*('Quantity', 'UnitPrice')).show(5)

+----------+-------------+------------------+
customerID|sum(Quantity)| sum(UnitPrice)|
+----------+-------------+------------------+
 17420| 265|116.43999999999997|
 16861| 133|18.730000000000004|
 16503| 567| 361.08|
 15727| 3065| 980.9399999999999|
 17389| 7430| 1273.57|
+----------+-------------+------------------+
only showing top 5 rows

`count()` is a bit different, it doesn't apply to any column, it count the number of rows in the DataFrame.  
This is different from pandas, where `.count()` count the number of non-null values.

In [ ]:
sales.groupBy('customerID').count().show(5)

+----------+-----+
customerID|count|
+----------+-----+
 17420| 30|
 16861| 8|
 16503| 86|
 15727| 302|
 17389| 224|
+----------+-----+
only showing top 5 rows

Other aggregation functions include: `.max()`, `.min()` and `.pivot()`.

And a last one: `.agg()`. This one is a bit meta, it will compute the aggregate of a function given as parameter.  
From the [Documentation](https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.GroupedData.agg)

> The available aggregate functions are `avg`, `max`, `min`, `sum`, `count`.

1. If exprs is a single dict mapping from string to string, then the key is the column to perform aggregation on, and the value is the aggregate function.

In [ ]:
agg_dict = {'Quantity': 'mean', 'UnitPrice': 'sum'}
sales.groupBy('customerId').agg(agg_dict).show(5)

+----------+------------------+------------------+
customerId| sum(UnitPrice)| avg(Quantity)|
+----------+------------------+------------------+
 17420|116.43999999999997| 8.833333333333334|
 16861|18.730000000000004| 16.625|
 16503| 361.08| 6.593023255813954|
 15727| 980.9399999999999|10.149006622516556|
 17389| 1273.57|33.169642857142854|
+----------+------------------+------------------+
only showing top 5 rows

Can use `'*'` if calling `.count()`. But not with others.

In [ ]:
sales.groupBy('customerId').agg({'*': 'count'}).show(5)

+----------+--------+
customerId|count(1)|
+----------+--------+
 17420| 30|
 16861| 8|
 16503| 86|
 15727| 302|
 17389| 224|
+----------+--------+
only showing top 5 rows

2. Alternatively, exprs can also be a "list" of aggregate Column expressions.  
This requires ***unpacking**

In [ ]:
agg_exprs = [F.mean('Quantity'), F.sum('UnitPrice')]
sales.groupBy('customerId').agg(*agg_exprs).show()

+----------+------------------+------------------+
customerId| avg(Quantity)| sum(UnitPrice)|
+----------+------------------+------------------+
 17420| 8.833333333333334|116.43999999999997|
 16861| 16.625|18.730000000000004|
 16503| 6.593023255813954| 361.08|
 15727|10.149006622516556| 980.9399999999999|
 17389|33.169642857142854| 1273.57|
 15447| 9.444444444444445|28.090000000000003|
 14450| 6.025|111.62000000000002|
 15100| 9.666666666666666| 65.7|
 12471| 15.0| 2266.869999999999|
 16916| 3.076923076923077| 317.52|
 14514| 8.068493150684931| 235.5300000000001|
 16500|11.181818181818182| 188.32|
 12626|10.912052117263844| 871.9999999999995|
 18161| 10.0| 294.16|
 15967| 2.511904761904762|240.88000000000002|
 17809| 31.484375| 897.1900000000002|
 15738|16.857954545454547|372.82000000000005|
 17044| 48.7|38.419999999999995|
 12393| 12.75| 145.9|
 16791| 2.357142857142857| 943.13|
+----------+------------------+------------------+
only showing top 20 rows

Now, you can alias, and that's because `.alias()` also returns a _Column Expression_.

In [ ]:
type(F.mean('Quantity').alias('meanQuantity'))

Out[20]: pyspark.sql.column.Column

In [ ]:
agg_exprs = [F.mean('Quantity'), F.sum('UnitPrice')]
sales.groupBy('customerId').agg(
    F.mean('Quantity').alias('meanQuantity'),
    F.sum('UnitPrice').alias('totalPrice')
).show(5)

+----------+------------------+------------------+
customerId| meanQuantity| totalPrice|
+----------+------------------+------------------+
 17420| 8.833333333333334|116.43999999999997|
 16861| 16.625|18.730000000000004|
 16503| 6.593023255813954| 361.08|
 15727|10.149006622516556| 980.9399999999999|
 17389|33.169642857142854| 1273.57|
+----------+------------------+------------------+
only showing top 5 rows